<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli and Rajath Kumar Mysore Pradeep Kumar</td>
</table>

Note:  This material largely follows the text "Numerical Linear Algebra" by Trefethen and Bau (SIAM, 1997) and is meant as a guide and supplement to the material presented there.

In [ ]:
from __future__ import print_function

%precision 6 
%matplotlib inline
import numpy
import matplotlib.pyplot as plt


# Understanding the Singular Value Decomposition



## Factorizations:  A review

**Matrix Factorization** is a fundamental concept in Linear Algebra and refers to the ability to write general matrices as the products of simpler matrices with special properties.

In particular, all of the great algorithms encountered in numerical linear Algebra can be succinctly described by their factorizations.  Examples include

### **The LU decomposition**  
$$PA=LU$$
    
* **Algorithms** *Gaussian Elimination with partial pivoting* 
* **Factorization** $P$ Permutation matrix (from pivoting), $L,U$: Lower and upper triangular matrices
* **Application** direct solution of $A\mathbf{x}=\mathbf{b}$ for $A\in\mathbb{R}^{n\times n}$

### **The QR decomposition**  
$$A=QR$$
    
* **Algorithms** Gram-Schmidt Orthogonalization, Householder Triangularization 
* **Factorization**  $Q$:  Orthonormal Basis for $C(A)$, $R$: upper triangular matrix
* **Applications** Least-Squares solutions, Projection problems,  Eigenvalues $QR/RQ$

### **EigenProblems**  
$$ AX = X\Lambda, \quad (A\in\mathbb{R}^{n\times n})$$

* Diagonalizable matrices $A = X\Lambda X^{-1}$
* Hermitian/Symmetric Matrices $A = Q\Lambda Q^T$
* Schur Factorization $A = QTQ^T$
    
    
* **Algorithms**: Power Method, Inverse Power with shifts,  $QR/RQ$
* **Factorization**-- $X$, $Q$: matrix of Eigenvectors,  $\Lambda$ and diagonal matrix of eigenvalues 
* **Application** Solution of Dynamical systems,  Iterative Maps, Markov Chains,  Vibrational analysis$\ldots$, Quantum Mechanics

*But perhaps the most beautiful factorization, which contains aspects of all of these problems$\ldots$

## Singular Value Decomposition (SVD)
$$
    A = U\Sigma V^{T}
$$
for *any* $A\in\mathbb{R}^{m\times n}$, where
* $U \in \mathbb R^{m \times m}$ and is the orthogonal matrix whose columns are the eigenvectors of $AA^{T}$
* $V \in \mathbb R^{n \times n}$ and is the orthogonal matrix whose columns are the eigenvectors of $A^{T}A$
* $\Sigma \in \mathbb R^{m \times n}$ and is a diagonal matrix with elements $\sigma_{1}, \sigma_{2}, \sigma_{3}, ... \sigma_{r}$ where $r = rank(A)$ corresponding to the square roots of the eigenvalues of $A^{T}A$. They are called the singular values of $A$ and are positive arranged in descending order. ($\sigma_{1} \geq \sigma_{2} \geq \sigma_{3} \geq ... \sigma_{r} > 0$).



Or a picture is worth a thousand words...


### Existence and Uniqueness

Every matrix $A \in \mathbb{C}^{m \times n}$ has a singular value decomposition. Furthermore, the singular values $\{\sigma_{j}\}$ are uniquely determined, and if $A$ is square and the $\sigma_{j}$ are distinct, the left and right singular vectors $\{u_{j}\}$ and $\{v_{j}\}$ are uniquely determined up to complex signs (i.e., complex scalar factors of absolute value 1).

### Eigenvalue Decomposition vs. SVD Decomposition

Let the matrix $X$ contain the eigenvectors of $A$ which are linearly independent, then we can write a decomposition of the matrix $A$ as
$$
    A = X \Lambda X^{-1}.
$$

How does this differ from the SVD?
 - The basis of the SVD representation differs from the eigenvalue decomposition
   - The basis vectors are not in general orthogonal for the eigenvalue decomposition where it is for the SVD
   - The SVD effectively contains two basis sets.
 - All matrices have an SVD decomposition whereas not all have eigenvalue decompositions.

#### Applications
* Matrix Pseudo-Inverse for ill-conditions problems
* Principal Component analysis
* Total Least Squares
* Image Compression
* Data Analysis for interpretation of high-dimensional data
* and more$\ldots$

### A conceptual computation of the SVD (not how it's really done)

Given $A = U\Sigma V^T$, we can form the matrix 

$$
    A^TA = V\Sigma^T\Sigma V^T
$$

where $V\in\mathbb{R}^{n\times n}$ is unitary and
$$
    \Sigma^T\Sigma = \begin{bmatrix} 
                \sigma_1^2 & & &    \\
                    & \sigma_2^2 & & \\
                   & & \ddots &   \\
                   & & &   \sigma_n^2  \\ 
                                   \end{bmatrix}
$$ 

However, because $A^TA$ is clearly **Symmetric** it can also has the eigen factorization

$$
    A^TA = Q\Lambda Q^T
$$

where $Q$ is unitary and $\Lambda$ is real diagonal.  

Moreover, we can show that $A^T A$ is at least Positive semi-definite so all the eigenvalues are $\geq 0$. 

In particular,  if $\mathrm{rank}(A)=r$ we know that $r$ eigenvalues are $>0$ and $n-r$ eigenvalues are equal to zero.

Given that

$$\begin{align}
    A^TA &= V\Sigma^T\Sigma V^T \\
        &= Q\Lambda Q^T \\
\end{align}
$$

If we simply order the eigenvalues from largest to smallest (and rearrange the columns of $Q$ to match), then we can make the association

$$V = Q\quad, \Sigma^T\Sigma = \Lambda$$ or

$$\sigma_i^2 = \lambda_i\quad\textrm{or}\quad \sigma_i = \sqrt{\lambda_i}$$

A similar argument can be made about $U$ and the eigenvectors of $AA^T$,  however, a better way to think about the relationship between $U$, $V$ and $\Sigma$ is to rearrange the SVD as

$$ 
    AV = U\Sigma
$$

or we can look at this column-by-column 

$$
    A\mathbf{v}_i = \sigma_i\mathbf{u}_i
$$

(in the same way the Eigen decomposition $AX = X\Lambda$ is equivalent to $A\mathbf{x}_i = \lambda_i\mathbf{x}_i$)

or rearranging 
$$
    \mathbf{u}_i = \frac{A\mathbf{v}_i}{\sigma_i},\quad i=1,2,\ldots,r
$$
lets us solve for the first $r$ columns of $U$ given $V$ and $\Sigma$.  

As we'll show, we usually don't need to solve for the remaing $m-r$ columns of $U$ for reasons that relate to the "4 fundamental subspaces of $A$"

### The Strangian view of the 4 Subspaces: a quick refresher

<img align=center src="./images/Strang_4_subspaces.png">

### The Big idea:  The columns of $U$ and $V$ contain orthonormal bases for the 4 fundamental subspaces

Returning to $AV = U\Sigma$ it follows that 

$$
\begin{align}
        A\mathbf{v}_i &= \sigma_i\mathbf{u_i},\quad\mathrm{for}\quad i=1,2\ldots r \\
        A\mathbf{v}_i &= \mathbf{0},\quad\mathrm{for}\quad i=r+1,\ldots n \\
\end{align}
$$

Therefore the last $n-r$ columns of $V$ must be in the Null space of $A$, and in fact must form an orthonormal basis for the Null space $N(A)$.  

Since the first $r$ columns of $V$ are all orthogonal to the last $n-r$ columns,  they must form an orthonormal basis for the row space $C(A^T)$.

Likewise,  since the first $r$ columns of $U$ satisfy

$$
    \mathbf{u}_i = \frac{A\mathbf{v}_i}{\sigma_i},\quad i=1,2,\ldots,r
$$

they must form an orthonormal basis for ??

Leaving only the last $m-r$ columns of $U$ as an orthonormal basis for the left null space $N(A^T)$

Again a picture is worth a lot here

$$
A\begin{bmatrix} &  &  & |&  &  &  \\
 &  &  & |&  &  &  \\
 \mathbf{v}_1 & \cdots & \mathbf{v}_r & | &\mathbf{v}_{r+1} & \cdots & \mathbf{v}_n \\
 &  &  & |&  &  &  \\
  &  &  & |&  &  &  \\
   \end{bmatrix} = \begin{bmatrix} &  &  & |&  &  &  \\
 &  &  & |&  &  &  \\
 &  &  & |&  &  &  \\
 \mathbf{u}_1 & \cdots & \mathbf{u}_r & | &\mathbf{u}_{r+1} & \cdots & \mathbf{u}_m \\
 &  &  & |&  &  &  \\
 &  &  & |&  &  &  \\
  &  &  & |&  &  &  \\
    \end{bmatrix} \begin{bmatrix} \sigma_1 &  &  & |  &  &    \\
 & \ddots &  & |   &   \mathbf{0}  \\
 &  & \sigma_r &|    &     \\
 -&- &-  & | & - &  -\\
 & \mathbf{0} &  & |   & \mathbf{0}   \\
 &  &  & |  &    \\
  &  &  & |  &     \\
    \end{bmatrix} 
$$


## The Economy (or Skinny) SVD

As it turns out, because of all the 0's  all we actually need to reconstruct $A$ is the the first $r$ columns of $U$, and $V$,  and the square sub-block of $\Sigma$ with just the singular values.  i.e.

$$
A\begin{bmatrix} &  &  & \\
 &  &  &    \\
 \mathbf{v}_1 & \cdots & \mathbf{v}_r   \\
 &  &  &   \\
  &  &  &    \\
   \end{bmatrix} = \begin{bmatrix} &  &  &   \\
 &  &  &    \\
  &  &  &    \\ 
 \mathbf{u}_1 & \cdots & \mathbf{u}_r     \\
 &  &  &    \\
 &  &  &    \\
  &  &  &    \\
    \end{bmatrix} \begin{bmatrix} \sigma_1 &  &        \\
 & \ddots &    \\
 &  & \sigma_r       \\
    \end{bmatrix} 
$$
or

$$
    A = U_r\Sigma_r V_r^T
$$

And it is this object (The Economy (or Skinny) SVD), that helps us understand what the SVD does.

### Spectral Theorem

The Economy SVD can also be written as 
$$
    A = U_r\Sigma_r V_r^T = \sum_{i=1}^r \sigma_i \mathbf{u}_i\mathbf{v}^T_i
$$

Which says we can expand $A$ as a series of rank-1 matrices $\mathbf{u}_i\mathbf{v}^T_i$ weighted by the singular values...

and it is this picture that leads to many of the important approximating properties of the SVD.

### Matrix Multiplication 

Consider the Matrix Vector product $A\mathbf{x}$ which maps a vector $\mathbf{x}$ in the row space $C(A^T)$ to its image in column space $C(A)$.  In the context of the Economy SVD
$$
    A\mathbf{x} = U_r\Sigma_r V_r^T\mathbf{x}
$$
or (since $V_r^TV_r = I$)

$$
\begin{align}
    A\mathbf{x} &= U_r\Sigma_r V_r^T\mathbf{x} \\
    &= U_r\Sigma_r (V_r^T V_r) V_r^T\mathbf{x} \\
    & = (U_r\Sigma_r V_r^T) (V_r V_r^T)\mathbf{x} \\
    & = A \mathbf{x}^+ 
\end{align}
$$
where $$\mathbf{x}^+ = (V_r V_r^T)\mathbf{x}$$
is ??

**Question**: If we wanted it, how would we  find the projection of $\mathbf{x}$ onto the Null space of $A$?

### SVD and the Matrix Inverse 

If a matrix is square and invertible, it implies that $m=n=r$ and $U$, $\Sigma$ and $V$ are all square invertible matrices,  therefore if 

$$
    A = U\Sigma V^T
$$
then

$$
    A^{-1} = V\Sigma^{-1} U^T
$$
where 
$$
    \Sigma^{-1} = \begin{bmatrix} 1/\sigma_1 &  &    &    \\
 & 1/\sigma_2 &   & \\
  &   &  \ddots &  \\
 &  &  & 1/\sigma_n       \\
    \end{bmatrix} 
$$

and clearly 
$$
    A^{-1}\mathbf{b} = \mathbf{x}
$$ 
maps any vector $\mathbf{b}\in C(A)$ back to a unique vector $\mathbf{x}\in C(A^T)$

### SVD and the Matrix Pseudo-Inverse 

But suppose $A$ is not square.  Then it cannot have an inverse,  however, the SVD allows us to define a "Pseudo-Inverse"

Given, the skinny SVD

$$
    A = U_r\Sigma_r V_r^T
$$

we can define

$$
    A^{+} = V_r \Sigma_r^{-1} U_r^T
$$
because $\Sigma_r$ is square and invertible (but it's size is set by the rank $r$)

### Action of the  Pseudo-Inverse 

In general, if $A$ is not square,  the problem $A\mathbf{x} = \mathbf{b}$ has either no solution or an infinite number of solutions.  However, it always has a minimal least square's solution given by 
$$
    x^{+} = A^{+}\mathbf{b}
$$
which maps any vector $\mathbf{b}\in C(A)$ to a unique vector $\mathbf{x}^+\in C(A^T)$

As we did with $A\mathbf{x}$, we can consider the action of because $A^{+}$ on any vector $\mathbf{b}$ as a projection problem.
$$
\begin{align}
    A^{+}\mathbf{b} &= V_r\Sigma^{-1}_r U_r^T\mathbf{x} \\
    &= V_r\Sigma^{-1}_r (U_r^T U_r) U_r^T\mathbf{x} \\
    & = (V_r\Sigma^{-1}_r U_r^T) (U_r U_r^T)\mathbf{x} \\
    & = A^{+} \mathbf{p}  
\end{align}
$$
where $$\mathbf{p}  = (U_r U_r^T)\mathbf{b}$$
is ??

### The minimal Least-Squares solution

It should be clear that 
$$
    \mathbf{x}^+ = A^{+}\mathbf{b}
$$ 
lies entirely in the Row space of $A$ (Why?)

Therefore it has no component in the $N(A)$ and is the shortest solution to $A\hat{\mathbf{x}}=\mathbf{p}$ which is the least squares problem

### The Pseudo-Inverse and ill-conditioned problems

Technically, a square matrix is either invertible or singular,  however, in many real problems, a matrix can be close to singular or "ill-conditioned".

Again, the condition number of a matrix defined by an induced $p$ norm is
$$
    \kappa_p(A) = ||A||_p ||A^{-1}||_p
$$

In particular,  for $p=2$, it is easy to show (using the SVD) that for a square matrix
$$
    \kappa_2(A) =\frac{\sigma_1}{\sigma_n}
$$

and thus for a singular matrix with $r<n$, $\kappa_2 =\infty$.

###  ill-conditioned matrices

However, it is possible that while $\sigma_n>0$,  it is significantly smaller than $\sigma_1$, leading to a very large condition number.  In particular if the last $j$ singular values are very small, it suggest that the matrix has a "near Null Space",  with a basis defined by the last $j$ columns of $V$.  Thus, while strictly invertible,  if a direct solver is used to solve
$$
A\mathbf{x} =\mathbf{b}
$$ 
any component in the near null space will be amplified by $1/\sigma_j$ and completely pollute your answer.

However, a useful fix can be to approximate $A$ as a lower rank matrix 

$$A_k = U_k\Sigma_kV_k^T$$

which just keeps the first $k$ singular values (i.e. pretends $r=k$).  If chosen wisely,  The approximate solution
$$
    \mathbf{x^{+} = A^{+}_k\mathbf{b}
$$ 
can be considerably more accurate as it does not allow the near-null space to contaminate the solution.

# Examples

### Full SVD example

Consider the matrix
$$
    A = \begin{bmatrix} 
        2 & 0 & 3 \\
        5 & 7 & 1 \\
        0 & 6 & 2 
    \end{bmatrix}.
$$
Confirm the SVD representation using `numpy` functions as appropriate.

In [ ]:
A = numpy.array([
    [2.0, 0.0, 3.0],
    [5.0, 7.0, 1.0],
    [0.0, 6.0, 2.0]
])

U, sigma, V_T = numpy.linalg.svd(A, full_matrices=True)

In [ ]:
print('U:\n{}\n'.format(U))
print('Sigma:\n{}\n'.format(numpy.diag(sigma)))
print('V:\n{}\n'.format(V_T.T))

In [ ]:
Aprime = numpy.dot(U, numpy.dot(numpy.diag(sigma), V_T))
print('A - USV^T:\n{}'.format(A - Aprime))

#### Example: a rank-1 Matrix 

$$A = \mathbf{x}\mathbf{y}^T$$

In [ ]:
x = numpy.array([ 1., 2., 1. ])
y = numpy.array([2., -1., 1. ])
A = numpy.outer(x,y)
print('A:\n{}\n'.format(A))
U, sigma, V_T = numpy.linalg.svd(A, full_matrices=False)

In [ ]:
print('U:\n{}\n'.format(U))
print('sigma:\n{}\n'.format(numpy.diag(sigma)))
print('V:\n{}\n'.format(V_T.T))

In [ ]:
Aprime = numpy.dot(U, numpy.dot(numpy.diag(sigma), V_T))
print('A - USV^T:\n{}'.format(A - Aprime))

In [ ]:
# rank-1 reconstruction
k = 1
A1 = numpy.dot(U[:,:k], numpy.dot(numpy.diag(sigma[:k]), V_T[:k,:]))
print('A_k =\n{}\n'.format(A1))
print('A - A_k:\n{}'.format(A - A1))

### An ill-conditioned example

Consider the matrix
$$
    A = \begin{bmatrix} 
        1 & 0 & 1 \\
        1 & 1 & 2+\epsilon \\
        0 & 1 & 1\\
    \end{bmatrix}.
$$
where the last column is almost the sum of the first two.

In [ ]:
epsilon = 1.e-3
A = numpy.array([
    [1.0, 0.0, 1.0],
    [1.0, 1.0, 2+epsilon],
    [0.0, 1.0, 1.0]
])

U, S, V_T = numpy.linalg.svd(A, full_matrices=True)

In [ ]:
print('U:\n{}\n'.format(U))
print('S:\n{}\n'.format(numpy.diag(S)))
print('V:\n{}\n'.format(V_T.T))
print('sigma_1/sigma_n = {}, cond_2(A) = {}'.format(S[0]/S[2], numpy.linalg.cond(A, p=2)))

### Try to solve $A\mathbf{x}=\mathbf{b}$

In [ ]:
finfo = numpy.finfo(numpy.float64)
epsilon = 2*finfo.eps
A = numpy.array([
    [1.0, 0.0, 1.0],
    [1.0, 1.0, 2+epsilon],
    [0.0, 1.0, 1.0]
])
U, S, VT = numpy.linalg.svd(A)
x_true = numpy.array([1., 2., 3.])
b = A.dot(x_true)
x = numpy.linalg.solve(A,b)
print('x = {}: cond(A)={}, S ={}'.format(x, numpy.linalg.cond(A, p=2), S))

In [ ]:
# rank-2 reconstruction
k = 2
A2 = numpy.dot(U[:,:k], numpy.dot(numpy.diag(S[:k]), VT[:k,:]))
#Pseudo Inverse
Ap2 = numpy.dot(VT.T[:,:k], numpy.dot(numpy.diag(1./S[:k]), U.T[:k,:]))
xp = Ap2.dot(b)
print('x = {}: cond(A)={}, S ={}'.format(x, numpy.linalg.cond(A, p=2), S))
print('x^+ = {}'.format(xp))

### Matrix Properties via the SVD

 - The $\text{rank}(A) = r$ where $r$ is the number of non-zero singular values.
 - The $\text{range}(A) = \mathrm{span}\langle u_1, ... , u_r\rangle$ and $\text{null}(a) = \mathrm{span}\langle v_{r+1}, ... , v_n\rangle$.
 - The $|| A ||_2 = \sigma_1$ and $||A||_F = \sqrt{\sigma_{1}^{2}+\sigma_{2}^{2}+...+\sigma_{r}^{2}}$.
 - The nonzero singular values of A are the square roots of the nonzero eigenvalues of $A^{T}A$ or $AA^{T}$.
 - If $A = A^{T}$, then the singular values of $A$ are the absolute values of the eigenvalues of $A$.
 - For $A \in \mathbb{C}^{m \times m}$ then $|det(A)| = \Pi_{i=1}^{m} \sigma_{i}$

### Low-Rank Approximations

 - $A$ is the sum of the $r$ rank-one matrices:
$$
    A = U \Sigma V^T = \sum_{j=1}^{r} \sigma_{j}u_{j}v_{j}^{T}
$$
 - For any $k$ with $0 \leq k \leq r$, define
$$
    A = \sum_{j=1}^{k} \sigma_{j}u_{j}v_{j}^{T}
$$
Let $k = min(m,n)$, then

$$
    ||A - A_{v}||_{2} = \text{inf}_{B \in \mathbb{C}^{m \times n}} \text{rank}(B)\leq k|| A-B||_{2} = \sigma_{k+1}
$$

- For any $k$ with $0 \leq k \leq r$, the matrix $A_{k}$ also satisfies
$$
    ||A - A_{v}||_{F} = \text{inf}_{B \in \mathbb{C}^{m \times n}} \text{rank}(B)\leq v ||A-B||_{F} = \sqrt{\sigma_{v+1}^{2} + ... + \sigma_{r}^{2}}
$$

#### Example:   "Hello World"

How does this work in practice?

In [ ]:
data = numpy.zeros((15,40))
m, n = data.shape
print('{}x{} pixel image'.format(m,n))


#H
data[2:10,2:4] = 1
data[5:7,4:6] = 1
data[2:10,6:8] = 1

#E
data[3:11,10:12] = 1
data[3:5,12:16] = 1
data[6:8, 12:16] = 1
data[9:11, 12:16] = 1

#L
data[4:12,18:20] = 1
data[10:12,20:24] = 1

#L
data[5:13,26:28] = 1
data[11:13,28:32] = 1

#0
data[6:14,34:36] = 1
data[6:8, 36:38] = 1
data[12:14, 36:38] = 1
data[6:14,38:40] = 1

plt.imshow(data)
plt.show()

In [ ]:
u, s, vt = numpy.linalg.svd(data)
print(u.shape, s.shape, vt.shape)


In [ ]:
fig = plt.figure(figsize=(8,6))
axes = fig.add_subplot(1, 1, 1)
axes.semilogy(s,'bo-')
axes.set_ylabel('$\sigma$', fontsize=16)
axes.grid()
axes.set_title('Spectrum of Singular Values')
plt.show()
print('s = {}'.format(s))


In [ ]:
fig = plt.figure()
fig.set_figwidth(fig.get_figwidth() * 3)
fig.set_figheight(fig.get_figheight() * 4)
for i in range(m):
    mode = s[i]*numpy.outer(u[:,i], vt[i,:])
    
    axes = fig.add_subplot(5, 3, i+1)
    mappable = axes.imshow(mode, vmin=0.0, vmax=1.0)
    axes.set_title('mode$_{{{}}}$'.format(i+1), fontsize=18)
    
plt.show()

In [ ]:
fig = plt.figure()
fig.set_figwidth(fig.get_figwidth() * 3)
fig.set_figheight(fig.get_figheight() * 4)
for k in range(1,16):
    Ak = u[:,:k].dot(numpy.diag(s[:k]).dot(vt[:k,:]))
    axes = fig.add_subplot(5, 3, k)
    mappable = axes.imshow(Ak, vmin=0.0, vmax=1.0)
    axes.set_title('$A_k$, $k={}$'.format(k))
    
plt.show()

### A bigger example:  the original Matrix (From Durer's Melancholia)

<img align=center src="./images/Durer_Melancholia_I.jpg" width=500>

In [ ]:
from matplotlib import image
data = image.imread('images/melancholia-magic-square.png')
m,n = data.shape
print('{}x{} pixel image'.format(m,n))
plt.figure(figsize=(8,6))
plt.imshow(data,cmap='gray')
plt.show()

In [ ]:
u, s, vt = numpy.linalg.svd(data)

In [ ]:
fig = plt.figure(figsize=(8,6))
axes = fig.add_subplot(1, 1, 1)
axes.semilogy(s,'bo-')
axes.set_ylabel('$\sigma$', fontsize=16)
axes.grid()
axes.set_title('Spectrum of Singular Values')
plt.show()

### First 4 Modes

In [ ]:
fig = plt.figure()
fig.set_figwidth(fig.get_figwidth() * 4)
fig.set_figheight(fig.get_figheight() * 1)
for i in range(4):
    mode = s[i]*numpy.outer(u[:,i], vt[i,:])

    axes = fig.add_subplot(1, 4, i+1)
    mappable = axes.imshow(mode, cmap='gray')
    axes.set_title('Mode = {}, $\sigma$={:3.3f}'.format(i+1,s[i]),fontsize=18)
    
plt.show()

### Reconstructions

In [ ]:
fig = plt.figure()
fig.set_figwidth(fig.get_figwidth() * 3)
fig.set_figheight(fig.get_figheight() *2)
for i,k in enumerate([1, 10, 20, 50, 100, 200]):
    Ak = u[:,:k].dot(numpy.diag(s[:k]).dot(vt[:k,:]))
    storage = 100.*k*(m+n + 1)/(m*n)
    axes = fig.add_subplot(2, 3, i+1)
    mappable = axes.imshow(Ak, vmin=0.0, vmax=1.0, cmap='gray')
    axes.set_title('$A_{{{}}}$, $storage={:2.2f}\%$'.format(k,storage),fontsize=16)
    
plt.show()

### Other Applications

* Total Least-squares
* PCA